# The Parity Problem
You are given a function as a black box: $ f: \{0, 1\}^n \rightarrow \{0, 1\}$ and $f(x) = u.x \mod2$.

For some hidden $u \in \{0, 1\}^n$, find $u$ with as few queries as possible. This is a restricted version of the Deutsch–Jozsa algorithm where instead of distinguishing between two different classes of functions, it tries to learn a string encoded in a function. 

## A classical solution

We need $n$ queries to compute the hidden string, because each time you run this circuit, you get only one bit of information. 
For example: let the hidden string $u = 101 $.

$$ f(100) = 1 $$ 

$$ f(010) = 0 $$

$$ f(001) = 1 $$

One possible solution to implement this classically is to use a classical AND gate as a mask to get each bit of information. 

## The Bernstein-Vazirani Algorithm
In contrast to the classical solution, the quantum solution requires only one query to find the hidden $u$. This algorithm was invented by Ethan Bernstein and Umesh Vazirani in 1992. Your task is to implement this algorithm in Q#. You can read more about the algorithm [here](https://en.wikipedia.org/wiki/Bernstein%E2%80%93Vazirani_algorithm).

The Bernstein-Vazirani algorithm is described as follows:
1. Initialize n query qubits in the state $|0\rangle$ and one answer qubit in the state $|-\rangle$
2. Apply the Hadamard Transform to an n-qubit query state to get $H|0\rangle^{\otimes n} =  \frac{1}{2^{n/2}} \sum_x |x> $.
3. Apply the oracle $U_f$ which transforms the state to  $\frac{1}{2^{n/2}} \sum_x (-1)^{u.x} |x> $
4. Apply the Hadamard Transform to the query state after the oracle is applied.
5. Measure in the computational basis.

## Task 1
**Summary**

Prepare a qubit state in a uniform superposition

**Input**

*query*: an array of query qubits

**Output** 

Apply the hadamard transform to n qubits in state $|0\rangle$. 


In [4]:
operation WalshHadamardTransform(query: Qubit[]): Unit is Adj{
    for i in 0 .. Length(query) - 1 {
        H(query[i]);
    }
}

WalshHadamardTransform

## Task 2
**Summary**

Prepare the answer qubit state in the state $|-\rangle$

**Input**

*answer*: the answer qubit

In [145]:
operation PrepareAnswerQubit(answer: Qubit): Unit {
    H(answer);
    Z(answer);
}

PrepareAnswerQubit

## Task 3


**Summary**

Encode the secret string (supplied as a boolean array s) into a function $U_f$

**Input**: 
query: qubit array
answer: answer qubit
s: secret string as a boolean array


In [146]:
operation Uf(query : Qubit[], answer: Qubit, s: Bool[]): Unit {
    
    for i in 0 .. Length(s) - 1 {
            if s[i] == true {
                CNOT(query[i], answer);
            }
    }
}

Uf

## Task 4: Explain what gates you applied to create the encoded string as $U_f$ and why you applied them.

The only gate you have to apply is the CNOT gate. Whichever position has a 1 in the hidden string you apply the CNOT with the control on the corresponding position in the query qubits and the target on the answer qubit. This make it so that the phase kickback will only affect the qubits corresponding to a 1 in the hidden string.

## Task 5: Run the Bernstein-Vazirani Algorithm using operations defined above

1. Create the secret string, and the required qubits
1. Perform the Bernstein Vazirani algorithm by using the tasks above. 
1. Measure the resulting query qubits. 
1. Transform the query results into an integer array which should give you the secret string. Return this integer array.

**Notice the hadamard-oracle-hadamard structure of the circuit. You can use the within..apply pattern in Q\# to do this. (Lecture 8 programming slides, Slide\# 26 )** 

In [147]:
operation BernsteinVazirani () : Int [] {
    mutable result = [0, size=6]; //Change the size according to the size of your secret string
    use q = Qubit[6];
    use answer = Qubit();
    let s = [true, false, false, true, false, false];
    
    
    within {
        WalshHadamardTransform(q);
    } apply{
        Uf(q, answer, s);
    }
    
    for i in 0 .. Length(q) - 1 {
            let mes = M(q[i]);
            let bit = mes == Zero ? 0 | 1;
            set result w/= i <- bit;
    }
    
    ResetAll(q);
    Reset(answer);
    return result;
}

BernsteinVazirani

In [159]:
%simulate BernsteinVazirani

1, 0, 0, 1, 0, 0

I've somehow managed to design code that works half of the time. I get 100100 50% of the time, and 000000 the other 50%. Not sure how I've managed that.